In [1]:

from binarybeech.binarybeech import *


ModuleNotFoundError: No module named 'pandas'

In [ ]:

df = pd.read_csv("data/titanic.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic),:]
    

In [ ]:
gbt_titanic = GradientBoostedTree(df_titanic,"Survived")

In [ ]:
gbt_titanic.X_names

In [ ]:

gbt_titanic._initial_model()


In [ ]:
it=gbt_titanic._initial_tree()
it.tree.show()


In [ ]:
it.tree.root

In [ ]:

gbt_titanic.learning_rate=0.5
gbt_titanic.create_trees(20)


In [ ]:

gbt_titanic.predict(df_titanic.iloc[0])

In [ ]:
m = gbt_titanic._confusion_matrix(df_test)

In [ ]:

P = tr.CART._precision(m)
R = tr.CART._recall(m)
A = tr.CART._accuracy(m)
F = tr.CART._F1(P,R)

print(P,R,F,A)


